<img src="https://gallery.mailchimp.com/f98d5ac0a3fbbdcdda35136ab/images/2002af76-5fd4-4185-9d49-28558b6b8772.png">

# `sg-hdb-resale-abr` 
# Part 2: Model Training
What we have done so far is to extract data from the .csv files, do some preliminary transformation to the data, and then loading all of it into an SQLite database. The next step is to work towards creating a simple predictive model for us to predict the price of a resale HDB unit. The following cells will have us load data from the SQLite database, that we have previously created.

In [1]:
# Import relevant packages
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Initialise engine
sql_engine = create_engine('sqlite:///../data/processed/sg_hdb.db')
# Simple query to get the whole table
query = "SELECT * FROM sg_hdb_resale"
# Store result of query in a pandas dataframe
sg_hdb_resale_df = pd.read_sql_query(query,sql_engine)

In [ ]:
# NOTE: If the cell above did not work for you, you can uncomment the following lines to query from a database that has been prepared beforehand
# sql_engine = create_engine('sqlite:///../reference/sg_hdb.db')
# query = "SELECT * FROM sg_hdb_resale"
# sg_hdb_resale_df = pd.read_sql_query(query, sql_engine)

In [4]:
# Observe result of query executed
sg_hdb_resale_df

,id,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,216,new generation,3 ROOM,73.0,1976,2000-01,0,159000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO
1,1,215,new generation,3 ROOM,73.0,1976,2000-01,0,167000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO
2,2,218,new generation,3 ROOM,67.0,1976,2000-01,0,163000.0,07 TO 09,ANG MO KIO AVE 1,ANG MO KIO
3,3,304,new generation,4 ROOM,97.0,1977,2000-01,0,260000.0,04 TO 06,ANG MO KIO AVE 1,ANG MO KIO
4,4,221,improved,5 ROOM,127.0,1993,2000-01,0,408000.0,10 TO 12,ANG MO KIO AVE 1,ANG MO KIO
...,...,...,...,...,...,...,...,...,...,...,...,...
790638,790638,611,apartment,EXECUTIVE,142.0,1987,1999-12,0,456000.0,10 TO 12,YISHUN ST 61,YISHUN
790639,790639,324,apartment,EXECUTIVE,142.0,1988,1999-12,0,408000.0,01 TO 03,YISHUN CTRL,YISHUN
790640,790640,392,maisonette,EXECUTIVE,146.0,1988,1999-12,0,469000.0,07 TO 09,YISHUN AVE 6,YISHUN
790641,790641,356,maisonette,EXECUTIVE,146.0,1988,1999-12,0,440000.0,04 TO 06,YISHUN RING RD,YISHUN


We are first going to inspect the data types of the imported dataframe.

In [7]:
sg_hdb_resale_df.dtypes

id                       int64
block                   object
flat_model              object
flat_type               object
floor_area_sqm         float64
lease_commence_date      int64
month                   object
remaining_lease          int64
resale_price           float64
storey_range            object
street_name             object
town                    object
dtype: object

Now, we create 2 separate series containing the predictor and response values to train our model on.

In [10]:
sg_hdb_x = sg_hdb_resale_df['floor_area_sqm'].values
sg_hdb_y = sg_hdb_resale_df['resale_price'].values

In [ ]:
# Import the relevant packages
import numpy as np
import sklearn
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle

When training a model, we have to create a train-test split to check the accuracy/performance of the model.

In [13]:
import numpy as np
import sklearn

from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Pickle in Python is primarily used in serializing and deserializing a Python object structure. In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program state across sessions, or transport data over the network.
import pickle 

In [16]:
sg_hdb_x_train, sg_hdb_x_test, sg_hdb_y_train, sg_hdb_y_test = sklearn.model_selection.train_test_split(
    sg_hdb_x, sg_hdb_y,
    test_size=0.3,random_state=7
)

In [19]:
# Reshaping needed when using a single variable for predictor

# The above is consistent with numpy advice/error message, to use reshape(-1,1) for a single feature; i.e. single column

#     Reshape your data using array.reshape(-1, 1) if your data has a single feature

sg_hdb_x_train = sg_hdb_x_train.reshape(-1,1)
sg_hdb_x_test = sg_hdb_x_test.reshape(-1,1)


We are gonna create a simple linear regression model (a.k.a best fit line) from the dataset.

In [20]:
# Initialise model
lm = LinearRegression()
# Create model from the train sets
lm.fit(sg_hdb_x_train, sg_hdb_y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
# To observe the model's coefficients
print('Coefficient: \n X:', lm.coef_, '\n c: ', lm.intercept_)

Coefficient: 
 X: [3517.23969499] 
 c:  -49306.87348512688


Y = mX + C

After creating the model, we are going to evaluate its performance by pitting it against the test set.

First, we use the linear model to provide us with the predictions derived from the values in the test set.

In [22]:
sg_hdb_y_pred = lm.predict(sg_hdb_x_test)
sg_hdb_y_pred


array([298899.856319  , 277796.41814905, 246141.26089413, ...,
       147658.54943438, 189865.42577427, 327037.77387893])

Thereafter, we are going to calculate the errors, pitting the predicted values with actual historical values.

In [23]:
# Examine fitness of model
r2_score(sg_hdb_y_test,sg_hdb_y_pred)

0.3983775602167309

Well, we did little to create a good model, hence the low value of fitness.

For the sake of this exercise, let us just proceed and export (serialise) this model for deployment. Save the model under a name, for example like the one below: 'sg_hdb_lm_v1.pkl'

In [24]:
# Specify output location of model to be serialised
file_loc_name ='../models/sg_hdb_1m_v1.pkl'
pickle.dump(lm,open(file_loc_name,'wb'))

Here, we just do a quick test by loading the model and then doing a single prediction to it.

In [25]:
# Test loading saved model
loaded_model=pickle.load(open(file_loc_name,'rb'))
# Create a test value for test prediction
# Test value has to be contained in a numpy array format hence np.array
test_val = np.array(63)
# Reshaping value before feeding to .predict function
test_val_reshape=test_val.reshape(-1,1)
# Conduct prediction
result = loaded_model.predict(test_val_reshape)
# Print out result
print(result)


[172279.22729932]


Now that we have exported this model, time to create a simple API (Application Programming Interface) that allows us to use the model, potentially remotely. On to [deployment](./sg-hdb-part3-basic-deployment.ipynb)!